In [9]:
CONFIG = {
    'data_folder' : 'C:/Users/Николай/PycharmProjects/VKRecSys/data/',
    'custom_data_folder' : 'C:/Users/Николай/PycharmProjects/VKRecSys/custom_data/',
    'models_folder' : 'C:/Users/Николай/PycharmProjects/VKRecSys/B.Processing/Модели/',
    "results_folder" : 'C:/Users/Николай/PycharmProjects/VKRecSys/C.Results/',
    
    "model_path_1" : '_5.6.pth_fold_0',
    "model_path_2" : '_5.6.pth_fold_1',
    "model_path_3" : '_5.6.pth_fold_2',
    "model_path_4" : '_5.6.pth_fold_3',
    
    'test_path': 'test_pairs.csv', 
    "output_path" : '5.6_ensemble_predictions.csv',
    
    'train_path' : 'train_interactions.parquet',
    'items_meta_path' : 'av4_items_meta.parquet',
    'users_meta_path' : 'av4_users_meta.parquet',
    
    'user_emb_size' : 256, # 183404
    'item_emb_size' : 256, # 337727
    'source_emb_size' : 256, # 19613
    'torch_precision' : 40, # number of decimal places for printing numbers
        
    'DEVICE' : 'cuda',
    'SEED' : 42,
    'BATCH_SIZE' : 16384,
    'LR' : 0.001,
    'EPOCHS' : 3,
    'output_dim' : 3
    
}

In [2]:
import numpy as np
import torch
import torch.nn.functional as F
import pandas as pd
from tqdm import tqdm
import torch.nn as nn

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

test = pd.read_csv(f"{CONFIG['data_folder']}{CONFIG['test_path']}")

In [4]:
items_meta = pd.read_parquet(f"{CONFIG['custom_data_folder']}{CONFIG['items_meta_path']}", engine='pyarrow')
users_meta = pd.read_parquet(f"{CONFIG['custom_data_folder']}{CONFIG['users_meta_path']}", engine='pyarrow')
users_meta['gender'] = users_meta['gender'].replace({1: 0, 2: 1})
# Normalization of numeric features
users_meta['age'] = (users_meta['age'] - users_meta['age'].min()) / (users_meta['age'].max() - users_meta['age'].min())
items_meta['duration'] = (items_meta['duration'] - items_meta['duration'].min()) / (
items_meta['duration'].max() - items_meta['duration'].min())

In [5]:
# Model definition
class MLPModel(nn.Module):
    def __init__(self, 
                 input_dim, 
                 num_users=users_meta.index.nunique(), 
                 num_items=items_meta.index.nunique(), 
                 num_sources=items_meta['source_id'].nunique(),
                 output_dim=CONFIG['output_dim'],
                 dropout_rate=0.2):  # Добавлен параметр dropout_rate
        
        super(MLPModel, self).__init__()
        self.user_embedding = nn.Embedding(num_users, CONFIG['user_emb_size'])
        self.item_embedding = nn.Embedding(num_items, CONFIG['item_emb_size'])
        self.source_embedding = nn.Embedding(num_sources, CONFIG['source_emb_size'])

        self.fc1 = nn.Linear(input_dim, 1024)
        self.fc2 = nn.Linear(1024, 1024)
        self.fc3 = nn.Linear(1024, 512)
        self.fc4 = nn.Linear(512, 256)
        self.fc5 = nn.Linear(256, 256)
        self.fc6 = nn.Linear(256, 128)
        self.fc7 = nn.Linear(128, 128)
        self.fc8 = nn.Linear(128, 64)
        self.fc9 = nn.Linear(64, output_dim)
        
        self.gelu = nn.GELU()

    def forward(self, user_ids, item_ids, source_ids, embeddings, u, i):
        user_emb = self.user_embedding(user_ids)
        item_emb = self.item_embedding(item_ids)
        source_emb = self.source_embedding(source_ids)

        x = torch.cat((user_emb, item_emb, source_emb, embeddings, u, i), dim=1)
        
        x = self.gelu(self.fc1(x))
        
        x = self.gelu(self.fc2(x))
        
        x = self.gelu(self.fc3(x))
        
        x = self.gelu(self.fc4(x))
        
        x = self.gelu(self.fc5(x))
        
        x = self.gelu(self.fc6(x))
        
        x = self.gelu(self.fc7(x))
        
        x = self.gelu(self.fc8(x))
        
        x = self.fc9(x)
        return x

In [6]:
input_dim = 1 + 1 + 1 + CONFIG['user_emb_size'] + CONFIG['item_emb_size'] + CONFIG['source_emb_size'] + 32 + 24

In [11]:
# Оценивание каждой модели и усреднение результатов
model_paths = [CONFIG['model_path_1'], CONFIG['model_path_2'], CONFIG['model_path_3'], CONFIG['model_path_4']]
all_predictions = []

In [13]:
for model_path in model_paths:
    print(f"Загрузка модели: {model_path}")
    model = MLPModel(input_dim).to(device)
    model.load_state_dict(torch.load(f"{CONFIG['models_folder']}{model_path}")["model_state_dict"])
    model.eval()
    
    predictions = []
    num_samples = len(test)
    num_batches = (num_samples + CONFIG['BATCH_SIZE'] - 1) // CONFIG['BATCH_SIZE']
    
    with torch.no_grad():
        for batch_idx in tqdm(range(num_batches), desc=f"Оценивание {model_path}"):
            start_idx = batch_idx * CONFIG['BATCH_SIZE']
            end_idx = min(start_idx + CONFIG['BATCH_SIZE'], num_samples)
            batch = test.iloc[start_idx:end_idx]

            batch_user_ids = torch.tensor(batch['user_id'].values, dtype=torch.long, device=device)
            batch_item_ids = torch.tensor(batch['item_id'].values, dtype=torch.long, device=device)
            batch_source_ids = torch.tensor(items_meta.loc[batch['item_id'].values, 'source_id'].values, dtype=torch.long, device=device)
            
            users_features = users_meta.loc[batch['user_id'].values]  
            users_features = torch.tensor(users_features.values, dtype=torch.float32, device=device)
            
            items_features = items_meta.loc[batch['item_id'].values].drop(columns=['source_id', 'embeddings'])
            items_features = torch.tensor(items_features.values, dtype=torch.float32, device=device)
            
            item_indices = batch_item_ids.cpu().numpy()
            embeddings = torch.tensor(np.stack(items_meta.loc[item_indices, 'embeddings'].values), device=device, dtype=torch.float32)
            

            outputs = model(batch_user_ids, batch_item_ids, batch_source_ids, embeddings, users_features, items_features)
            
            probabilities = F.softmax(outputs, dim=1)

            # Взвешенные предсказания
            class_weights = torch.tensor([0, 1, 2], device=probabilities.device, dtype=probabilities.dtype)
            weighted_predictions = torch.sum(probabilities * class_weights, dim=1).cpu().numpy()

            predictions.extend(weighted_predictions)
    
    all_predictions.append(predictions)
    del model
    torch.cuda.empty_cache()

# Усреднение предсказаний
ensemble_predictions = np.mean(all_predictions, axis=0)

Загрузка модели: _5.6.pth_fold_0


C:\Users\Николай\AppData\Local\Temp\ipykernel_2352\3536639851.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"{CONFIG['models_folder']

Загрузка модели: _5.6.pth_fold_1


C:\Users\Николай\AppData\Local\Temp\ipykernel_2352\3536639851.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"{CONFIG['models_folder']

Загрузка модели: _5.6.pth_fold_2


C:\Users\Николай\AppData\Local\Temp\ipykernel_2352\3536639851.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"{CONFIG['models_folder']

Загрузка модели: _5.6.pth_fold_3


C:\Users\Николай\AppData\Local\Temp\ipykernel_2352\3536639851.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"{CONFIG['models_folder']

In [14]:
# Сохранение результатов
test['predict'] = ensemble_predictions
output_path = f"{CONFIG['results_folder']}{CONFIG['output_path']}"
test[['user_id', 'item_id', 'predict']].to_csv(output_path, index=False)
print(f"Результаты сохранены в {output_path}")

Результаты сохранены в C:/Users/Николай/PycharmProjects/VKRecSys/C.Results/5.6_ensemble_predictions.csv
